In [ ]:
import random
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
from collections import namedtuple
from scipy.sparse import csr_matrix
import grblas
grblas.init('suitesparse')
from grblas import Matrix, Vector
from grblas import descriptor
from grblas import UnaryOp, BinaryOp, Monoid, Semiring

In [ ]:
def vrepr(v):
    df = pd.DataFrame(index=range(v.size), columns=[''])
    for i, val in zip(*v.to_values()):
        df.iloc[i] = val
    return df.where(pd.notnull(df), '')

def hrepr(v):
    return vrepr(v).T

def mrepr(m):
    df = pd.DataFrame(columns=range(m.ncols), index=range(m.nrows))
    for i, j, val in zip(*m.to_values()):
        df.iloc[i, j] = val
    return df.where(pd.notnull(df), '')

def draw(m):
    g = nx.DiGraph()
    for row, col, val in zip(*m.to_values()):
        g.add_edge(row, col, weight=val)
    pos = nx.spring_layout(g)
    edge_labels = {(i, j): d['weight'] for i, j, d in g.edges(data=True)}
    nx.draw_networkx(g, pos, node_color='red', node_size=500)
    nx.draw_networkx_edge_labels(g, pos, edge_labels=edge_labels)

In [ ]:
def adj_matrix_to_grblas(m, dtype=grblas.dtypes.FP64):
    ss = csr_matrix(m)
    nrows, ncols = ss.shape
    rows, cols = ss.nonzero()
    g = grblas.Matrix.new_from_values(rows, cols, ss.data, nrows=nrows, ncols=ncols, dtype=dtype)
    return g

def nxgraph_to_grblas(G, dtype=grblas.dtypes.FP64):
    ss = nx.convert_matrix.to_scipy_sparse_matrix(G)
    nrows, ncols = ss.shape
    rows, cols = ss.nonzero()
    g = grblas.Matrix.new_from_values(rows, cols, ss.data, nrows=nrows, ncols=ncols, dtype=dtype)
    return g

# Louvain Community Detection

https://en.wikipedia.org/wiki/Louvain_modularity

This will return a list of progressively smaller graphs, sort of like zooming out in a map and having houses collapse into a neighborhood and neighborhoods collapse into cities

In [ ]:
def compute_modularity(adj, comms=None):
    """
    Given adjacency matrix (nxn) and community matrix (cxn), compute the modularity metric
    """
    nn = adj.nrows  # number of nodes in current graph
    if comms is None:
        # Build the identity matrix, assigning each node to its own community
        comms = grblas.Matrix.new_from_values(range(nn), range(nn), [1]*nn, nrows=nn, ncols=nn)
    nc = comms.nrows  # number of possible communities
    total_links_doubled = grblas.Scalar.new_from_type(adj.dtype)
    community_tmp = grblas.Matrix.new_from_type(adj.dtype, nrows=nc, ncols=nn)
    community = grblas.Matrix.new_from_type(adj.dtype, nrows=nc, ncols=nc)
    diag_mask = grblas.Matrix.new_from_values(range(nc), range(nc), [True]*nc, nrows=nc, ncols=nc, dtype=grblas.dtypes.BOOL)
    diag_matrix = grblas.Matrix.new_from_type(adj.dtype, nrows=nc, ncols=nc)
    diag_vector = grblas.Vector.new_from_type(adj.dtype, size=nc)
    modularity = grblas.Vector.new_from_type(adj.dtype, size=nc)
    
    total_links_doubled[:] = adj.reduce_scalar(grblas.Monoid.PLUS)
    community_tmp[:] = comms.mxm(adj)
    community[:] = community_tmp.mxm(comms.T)
    diag_matrix[diag_mask] = community
    diag_vector[:] = diag_matrix.reduce_columns()  # Aij
    modularity[:] = community.reduce_columns(grblas.BinaryOp.PLUS)  # ki
    modularity[:] = modularity.ewise_mult(modularity, grblas.BinaryOp.TIMES)  # ki^2
    modularity.assign[modularity, grblas.BinaryOp.DIV] = total_links_doubled  # ki^2/2m
    modularity.assign[modularity, grblas.BinaryOp.TIMES] = -1  # -ki^2
    modularity[:] = diag_vector.ewise_add(modularity, grblas.BinaryOp.PLUS)  # Aij - ki^2/2m
    modularity.assign[modularity, grblas.BinaryOp.DIV] = total_links_doubled  # (Aij - ki^2/2m) / 2m
    # Reuse scalar to compute sum of community modularity scores
    total_links_doubled[:] = modularity.reduce(grblas.Monoid.PLUS)  # (1/2m)*sum(Aij - ki^2/2m)
    
    return total_links_doubled

### General note about spec v1.2

We really need v\[:] = v.apply(grblas.BinaryOp.TIMES, right=2)

We can spell it like this:
v.assign\[v, grblas.BinaryOp.TIMES] = 2

But that requires itself as a mask and doesn't allow for the dtype to change.

For example, this is impossible:
w = grblas.Vector.new_from_type(grblas.dtypes.BOOL, v.size)
w[:] = v.apply(grblas.BinaryOp.EQ, right=2.7)

In [ ]:
def find_best_community(node, adj, comms):
    """
    Updates comms (community grouping) for node to reach max modularity
    Returns True if node changed community. False otherwise.
    """
    nc = comms.nrows  # number of possible communities
    node_vec = grblas.Vector.new_from_values([node], [1], size=nc)
    # Save current modularity score in current community
    current_community = comms.extract[:, node].new()
    current_community_index = next(current_community.to_values()[0])
    orig_modularity_score = compute_modularity(adj, comms)
    
    # Move node to its own community
    beyond_last_index = comms.nrows - 1
    beyond_last = grblas.Vector.new_from_values([beyond_last_index], [1], size=comms.nrows)
    comms.assign[:, node] = beyond_last
    baseline_modularity_score = compute_modularity(adj, comms)
    
    # Compute modularity improvements for each neighbor
    total_links_doubled = adj.reduce_scalar(grblas.Monoid.PLUS).new().value
    ki_all = adj.reduce_columns(grblas.Monoid.PLUS).new()
    ki = ki_all.element[node]
    communitized_tmp = grblas.Matrix.new_from_type(grblas.dtypes.FP64, nrows=nc, ncols=adj.nrows)
    communitized = grblas.Matrix.new_from_type(grblas.dtypes.FP64, nrows=nc, ncols=nc)
    communitized_tmp[:] = comms.mxm(adj)
    communitized[:] = communitized_tmp.mxm(comms.T)
    sigma_total = communitized.reduce_columns(grblas.Monoid.PLUS).new()
    ki_in = grblas.Vector.new_from_type(grblas.dtypes.FP64, size=nc)
    ki_in[~beyond_last] = beyond_last.vxm(communitized)
    # delta = 2*ki_in/total_links_doubled - 2*sigma_total*ki/total_links_doubled^2 - kloop/total_links_doubled
    delta = grblas.Vector.new_from_type(grblas.dtypes.FP64, size=nc)
    delta[:] = ki_in
    delta.assign[delta, grblas.BinaryOp.TIMES] = 2/total_links_doubled
    sigma_total.assign[sigma_total, grblas.BinaryOp.TIMES] = -2*ki/total_links_doubled**2
    delta[:] = delta.ewise_mult(sigma_total, grblas.BinaryOp.PLUS)
    
    # Choose best neighbor
    max_modularity_delta = delta.reduce(grblas.Monoid.MAX).new()
    
    # If modularity is improved, update comms and return True
    if max_modularity_delta.value > orig_modularity_score.value - baseline_modularity_score.value:
        max_mask = grblas.Vector.new_from_existing(delta)
        max_mask.assign[max_mask, grblas.BinaryOp.EQ] = max_modularity_delta
        delta[max_mask, grblas.REPLACE] = delta  # eliminate all but the max value(s)
        indexes, vals = delta.to_values()
        best_community = next(indexes)
        # Guard against reassigning a node to its existing community
        if best_community != current_community_index and best_community != beyond_last_index:
            comms.assign[:, node] = comms.extract[:, best_community].new()
            return True

    # If modularity isn't improved, reset and return False
    comms.assign[:, node] = current_community
    return False

In [ ]:
def optimize_communities(adj, max_iter=20):
    """
    Given an adjacency matrix `adj`, returns a compact community mapping of size cxn
    where c is the number of communities and n is the number of nodes in the graph (i.e. adj is nxn)
    The community mapping is done by repeatedly iterating over the nodes one by one
    to find the best community (defined as maximizing the modularity).
    Once an full pass over the nodes yields no changes, this will return.
    If max_iter is reached, it will also return.
    """
    n = adj.nrows
    comms = grblas.Matrix.new_from_values(range(n), range(n), [1]*n, nrows=n+1, ncols=n)
    for i_iter in range(max_iter):
        nodes = list(range(n))
        random.shuffle(nodes)
        comms_modified = False
        for node in nodes:
            node_moved = find_best_community(node, adj, comms)
            if node_moved:
                comms_modified = True
        if not comms_modified:
            break
    
    # Compact comms
    rows, cols, vals = comms.to_values()
    nonzero_rows = list(sorted(set(rows)))
    compact_comms = grblas.Matrix.new_from_type(comms.dtype, nrows=len(nonzero_rows), ncols=comms.ncols)
    compact_comms[:] = comms.extract[nonzero_rows, :]
    
    return compact_comms

In [ ]:
LouvainResult = namedtuple('LouvainResult', ['adj', 'cmap', 'modscore'])

def louvain_levels(adj, max_iter=20):
    """
    Returns a list of LouvainResult -- a namedtuple with
    - adj: adjacency matrix
    - cmap: community map matrix
    - modscore: modularity score
    Each item in the returned list represents one pass through the Louvain community detection algorithm.
    The size of the adjacency matrix should shrink while the modularity score should increase.
    """
    results = []
    adj_list = [adj]
    comm_list = []
    while True:
        modscore = compute_modularity(adj)
        comms = optimize_communities(adj, max_iter)
        results.append(LouvainResult(adj, comms, modscore))
        # Exit criteria: number of communities did not decrease
        if comms.nrows >= adj.nrows:
            break
        # Compress the adjacency graph
        nc = comms.nrows
        prev_adj = adj
        adj_tmp = grblas.Matrix.new_from_type(prev_adj.dtype, nrows=nc, ncols=prev_adj.nrows)
        adj = grblas.Matrix.new_from_type(prev_adj.dtype, nrows=nc, ncols=nc)
        adj_tmp[:] = comms.mxm(prev_adj)
        adj[:] = adj_tmp.mxm(comms.T)
        adj_list.append(adj)
        
    return results

## Example

- Nodes 0, 1, 3, 4 are fully connected
- Nodes 2, 5, 6 are fully connected
- There is a single connection between nodes 2 and 4 to connect the two groups
- All edges have a weight of 1

In [ ]:
m = np.array([[0,1,0,1,1,0,0],
              [1,0,0,1,1,0,0],
              [0,0,0,0,1,1,1],
              [1,1,0,0,1,0,0],
              [1,1,1,1,0,0,0],
              [0,0,1,0,0,0,1],
              [0,0,1,0,0,1,0]])
g = adj_matrix_to_grblas(m)

In [ ]:
l = louvain_levels(g)
l

In [ ]:
mrepr(l[1].adj)

In [ ]:
mrepr(l[0].cmap)

## Example

https://neo4j.com/docs/graph-algorithms/current/algorithms/louvain/#algorithms-louvain-examples-stream-intermediate

If the results don't match the website's clustering, try re-running.

In [ ]:
m = [
[0,1,0,1,0,1,0,0,0,0,0,0,0,0,0],
[1,0,0,1,1,0,1,0,0,0,0,0,0,0,1],
[0,0,0,0,0,1,0,0,0,0,0,0,0,0,1],
[1,1,0,0,0,0,0,0,0,0,1,0,0,0,0],
[0,1,0,0,0,1,0,1,0,0,0,0,0,0,1],
[1,0,1,0,1,0,1,0,0,0,0,0,0,0,0],
[0,1,0,0,0,1,0,1,0,0,0,0,0,0,0],
[0,0,0,0,1,0,1,0,1,1,0,0,0,0,0],
[0,0,0,0,0,0,0,1,0,0,1,0,0,0,0],
[0,0,0,0,0,0,0,1,0,0,1,0,1,1,0],
[0,0,0,1,0,0,0,0,1,1,0,1,1,0,0],
[0,0,0,0,0,0,0,0,0,0,1,0,0,1,0],
[0,0,0,0,0,0,0,0,0,1,1,0,0,1,0],
[0,0,0,0,0,0,0,0,0,1,0,1,1,0,0],
[0,1,1,0,1,0,0,0,0,0,0,0,0,0,0],
]
g = adj_matrix_to_grblas(m)

In [ ]:
l = louvain_levels(g)
l

In [ ]:
mrepr(l[0].cmap)

In [ ]:
mrepr(l[1].adj)

In [ ]:
mrepr(l[1].cmap)

In [ ]:
mrepr(l[2].adj)

## Example

Figure 1 from https://arxiv.org/pdf/0803.0476.pds

If the results don't match the paper, try running it again. Because the algorithm is non-deterministic, sometimes the communities don't match the results in the paper. But often they will.

In [ ]:
m = [
[0,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0],
[0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0],
[1,1,0,0,1,1,1,0,0,0,0,0,0,0,0,0],
[1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0],
[1,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0],
[1,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0],
[0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0],
[0,1,0,1,0,1,1,0,0,0,0,0,0,0,0,0],
[0,0,0,0,0,0,0,0,0,1,1,1,0,0,1,1],
[0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0],
[0,0,0,0,1,0,0,0,1,0,0,1,1,1,1,0],
[0,0,0,0,0,1,1,0,1,0,1,0,0,1,0,0],
[0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0],
[0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0],
[0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0],
[0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0],
]
g = adj_matrix_to_grblas(m)

In [ ]:
l = louvain_levels(g)
l

In [ ]:
mrepr(l[1].adj)

In [ ]:
mrepr(l[2].adj)